In [52]:
# 我们要爬全国各地每个城市的最低温度，并且可视化排名
import requests
from bs4 import BeautifulSoup
from pyecharts import Bar

ALL_DATA = []

def parse_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    text = response.content.decode('utf-8')
    # htm
    soup = BeautifulSoup(text, 'html5lib') #因为香港的html标签不规范#但是这个解析器比html要慢点
    conMidtab = soup.find('div', class_='conMidtab')
    tables = conMidtab.find_all('table')
    for table in tables: #每个table都是一个省/直辖市
        trs = table.find_all('tr')[2:]
        for index, tr in enumerate(trs): #每个tr都是一个城市
            tds = tr.find_all('td')
            city_td = tds[1] if index == 0 else tds[0] #因为第一个的城市名在第2个位置，其他的城市名在第1个位置
            city = list(city_td.stripped_strings)[0]
            temp_td = tds[-2]
            min_temp = list(temp_td.stripped_strings)[0]
            ALL_DATA.append({"city":city, "min_temp": int(min_temp)})
            

def main():
    regions = ['hb', 'db', 'hd', 'hz', 'hn', 'xb', 'xn', 'gat']
    for region in regions:
        #每个url都是一个地区
        url = 'http://www.weather.com.cn/textFC/{}.shtml'.format(region)
        parse_page(url)
    
    # 分析数据
    # 根据最低气温进行排序
    '''
    def sort_key(data):
        min_temp = data['min_temp']
        return min_temp
        
    ALL_DATA.sort(key = sort_key)
    '''
    # 相当于上面这个表达式
    ALL_DATA.sort(key = lambda data: data['min_temp'])
    data = ALL_DATA[0:10] #我们取温度最低的十个城市进行排序
    # data = ALL_DATA[-10:len(ALL_DATA)] 这样的话就是温度最高的十个城市
    '''
    cities =[]
    for city_temp in data:
        city = city_temp['city']
        cities.append(city)
    '''
    # 相当于上面这个表达式
    cities = list(map(lambda x: x['city'], data))
    temps = list(map(lambda x: x['min_temp'], data))
    chart = Bar("中国天气最低气温排行榜")
    chart.add("", cities, temps)
    # render可以渲染一个文件
    chart.render('temperature.html')
    
if __name__ == '__main__':
    main()
    
    
# 香港的table格式不规范，table只有开头没有结尾:</table>
# inspection里自动补充了，但是查看源代码，会发现没有补进去
# 我们可以用html5lib来解析（会自动把标签给补进去）